<a href="https://colab.research.google.com/github/a-shn/TweetsClassificationCNN/blob/master/%D0%A2%D0%B5%D1%81%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_CNN_%D0%BD%D0%B0_%D0%BA%D0%BE%D1%80%D0%BF%D1%83%D1%81%D0%B5_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Результаты в виде таблицы:**

С нормализацией, без лемматизации:

Train corpus | Test corpus | Precision | Recall | F-measure
--- | --- | --- | --- | ---
train | validation | 0.27559 | 0.78947 | 0.40856
train + validation | test | 0.29630 | 0.67470 | 0.41176
train + validation + test | test_final | 0.36881 | 0.75253 | 0.49502

Без нормализации, без лемматизации:

Train corpus | Test corpus | Precision | Recall | F-measure
--- | --- | --- | --- | ---
train | validation | 0.64705 | 0.24812 | 0.35869
train + validation | test | 0.51063 | 0.28915 | 0.36923
train + validation + test | test_final | 0.74698 | 0.31313 | 0.44128

Без нормализации, с лемматизацией:

Train corpus | Test corpus | Precision | Recall | F-measure
--- | --- | --- | --- | ---
train | validation | 0.60000 | 0.27068 | 0.37306
train + validation | test | 0.54000 | 0.16265 | 0.25000
train + validation + test | test_final | 0.72340 | 0.34343 | 0.46575

# **Считывание и разделение данных, инициализация TextCNN и импортирование fasttext-модели**

In [166]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [167]:
KEEP_COLUMNS = ["class", "tweet"]

training_raw_path = '/content/drive/My Drive/NLP/RuADReCT_raw/task2_ru_training_raw.tsv'
valiadtion_raw_path = '/content/drive/My Drive/NLP/RuADReCT_raw/task2_ru_validation_raw.tsv'
test_raw_path = '/content/drive/My Drive/NLP/RuADReCT_raw/task2_ru_test_raw.tsv'
test_final_raw_path = '/content/drive/My Drive/NLP/RuADReCT_raw/task2_ru_test_final_raw.tsv'
fasttext_model_path = '/content/drive/My Drive/NLP/rudrec_fasttext/rudrec_fasttext_model.bin'

In [168]:
# В зависимости от задания, откомментировать нужную часть

# 1 Задание:
# train_df = pd.read_csv(training_raw_path, sep="\t", encoding="utf-8")
# test_df = pd.read_csv(valiadtion_raw_path, sep="\t", encoding="utf-8")


# 2 Задание:
# train_df = pd.concat([pd.read_csv(training_raw_path, sep="\t", encoding="utf-8"), 
#                       pd.read_csv(valiadtion_raw_path, sep="\t", encoding="utf-8")])
# test_df = pd.read_csv(test_raw_path, sep="\t", encoding="utf-8")

# 3 Задание:
train_df = pd.concat([pd.read_csv(training_raw_path, sep="\t", encoding="utf-8"), 
                      pd.read_csv(valiadtion_raw_path, sep="\t", encoding="utf-8"),
                      pd.read_csv(test_raw_path, sep="\t", encoding="utf-8")])
test_df = pd.read_csv(test_final_raw_path, sep="\t", encoding="utf-8")

train_df = train_df[KEEP_COLUMNS] 
train_df = train_df[(train_df['class'] == 0) | (train_df['class'] == 1)] # Удаляем невалидные строки (класс NaN, например)
test_df = test_df[KEEP_COLUMNS]
test_df = test_df[(test_df['class'] == 0) | (test_df['class'] == 1)]

#P.S. в task2_ru_test_final_raw.tsv название колонки с твитами называется "text", а не "tweet", 
#как в других, поэтому перед запуском 3-его задания, нужно ее переименовать в "tweet"

In [169]:
train_df, dev_df, _, _ = \
    train_test_split(train_df, train_df, test_size=0.1, random_state=42)
train_positive_class_df = train_df[train_df['class'] == 1]
train_negative_class_df = train_df[train_df['class'] == 0]
num_min_examples = min(train_positive_class_df.shape[0], train_negative_class_df.shape[0])
train_positive_class_df = train_positive_class_df.sample(num_min_examples)
train_negative_class_df = train_negative_class_df.sample(num_min_examples)

class_normalized_train_df = pd.concat([train_positive_class_df, train_negative_class_df]).sample(frac=1)
train_df = class_normalized_train_df # для обучения без нормирования - закомментировать

In [ ]:
!pip install tensorflow
!pip install fasttext

In [171]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Embedding, Dense, Conv1D, GlobalMaxPooling1D, Concatenate

class TextCNN(Model):
    def __init__(self,
                 maxlen,
                 max_features,
                 embedding_dims,
                 kernel_sizes=[3, 4, 5],
                 class_num=1,
                 last_activation='sigmoid',
                 embedding_weights=None):
        super(TextCNN, self).__init__()
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.kernel_sizes = kernel_sizes
        self.class_num = class_num
        self.last_activation = last_activation
        self.embedding = Embedding(self.max_features, self.embedding_dims,
                                   input_length=self.maxlen, weights=[embedding_weights], )
        self.convs = []
        self.max_poolings = []
        for kernel_size in self.kernel_sizes:
            self.convs.append(Conv1D(128, kernel_size, activation='relu'))
            self.max_poolings.append(GlobalMaxPooling1D())
        self.classifier = Dense(self.class_num, activation=self.last_activation)

    def call(self, inputs):
        if len(inputs.get_shape()) != 2:
            raise ValueError('The rank of inputs of TextCNN must be 2, but now is %d' % len(inputs.get_shape()))
        if inputs.get_shape()[1] != self.maxlen:
            raise ValueError(
                'The maxlen of inputs of TextCNN must be %d, but now is %d' % (self.maxlen, inputs.get_shape()[1]))
        # Embedding part can try multichannel as same as origin paper
        embedding = self.embedding(inputs)
        convs = []
        for i in range(len(self.kernel_sizes)):
            c = self.convs[i](embedding)
            c = self.max_poolings[i](c)
            convs.append(c)
        x = Concatenate()(convs)
        output = self.classifier(x)
        return output

In [172]:
import fasttext
import numpy as np
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from keras_preprocessing import sequence
from keras_preprocessing.text import Tokenizer
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from tensorflow.keras.callbacks import EarlyStopping

In [173]:
BATCH_SIZE = 128
EMBEDDINGS_DIM = 200
CLASSIFIER_TRAIN_EPOCHS = 10
CLASSIFICATION_THRESHOLD = 0.5

In [174]:
# Loading pretrained fastext model
fasttext_model = fasttext.load_model(fasttext_model_path)

# Extracting tweet texts
train_tweet_texts = train_df.tweet.values
test_tweet_texts = test_df.tweet.values
dev_tweet_texts = dev_df.tweet.values

# Extracting tweet labels
train_labels = train_df['class'].values
test_labels = test_df['class'].values
dev_labels = dev_df['class'].values

# **Пре-процессинг:**

In [175]:
import re
def list_replace(search, replacement, text):
    """
    Replaces all symbols of text which are present
    in the search string with the replacement string.
    """
    search = [el for el in search if el in text]
    for c in search:
        text = text.replace(c, replacement)
    return text

def clean_text(text):

    text = list_replace \
        ('\u00AB\u00BB\u2039\u203A\u201E\u201A\u201C\u201F\u2018\u201B\u201D\u2019', '\u0022', text)

    text = list_replace \
        ('\u2012\u2013\u2014\u2015\u203E\u0305\u00AF', '\u2003\u002D\u002D\u2003', text)

    text = list_replace('\u2010\u2011', '\u002D', text)

    text = list_replace \
            (
            '\u2000\u2001\u2002\u2004\u2005\u2006\u2007\u2008\u2009\u200A\u200B\u202F\u205F\u2060\u3000',
            '\u2002', text)

    text = re.sub('\u2003\u2003', '\u2003', text)
    text = re.sub('\t\t', '\t', text)

    text = list_replace \
            (
            '\u02CC\u0307\u0323\u2022\u2023\u2043\u204C\u204D\u2219\u25E6\u00B7\u00D7\u22C5\u2219\u2062',
            '.', text)

    text = list_replace('\u2217', '\u002A', text)

    text = list_replace('…', '...', text)

    text = list_replace('\u00C4', 'A', text)
    text = list_replace('\u00E4', 'a', text)
    text = list_replace('\u00CB', 'E', text)
    text = list_replace('\u00EB', 'e', text)
    text = list_replace('\u1E26', 'H', text)
    text = list_replace('\u1E27', 'h', text)
    text = list_replace('\u00CF', 'I', text)
    text = list_replace('\u00EF', 'i', text)
    text = list_replace('\u00D6', 'O', text)
    text = list_replace('\u00F6', 'o', text)
    text = list_replace('\u00DC', 'U', text)
    text = list_replace('\u00FC', 'u', text)
    text = list_replace('\u0178', 'Y', text)
    text = list_replace('\u00FF', 'y', text)
    text = list_replace('\u00DF', 's', text)
    text = list_replace('\u1E9E', 'S', text)
    # Removing punctuation
    text = list_replace(',.[]{}()=+-−*&^%$#@!~;:§/\|\?"\n', ' ', text)
    # Replacing all numbers with masks
    text = list_replace('0123456789', 'x', text)

    currencies = list \
            (
            '\u20BD\u0024\u00A3\u20A4\u20AC\u20AA\u2133\u20BE\u00A2\u058F\u0BF9\u20BC\u20A1\u20A0\u20B4\u20A7\u20B0\u20BF\u20A3\u060B\u0E3F\u20A9\u20B4\u20B2\u0192\u20AB\u00A5\u20AD\u20A1\u20BA\u20A6\u20B1\uFDFC\u17DB\u20B9\u20A8\u20B5\u09F3\u20B8\u20AE\u0192'
        )

    alphabet = list \
            (
            '\t\r абвгдеёзжийклмнопрстуфхцчшщьыъэюяАБВГДЕЁЗЖИЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯabcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ ')

    allowed = set(currencies + alphabet)

    cleaned_text = [sym for sym in text if sym in allowed]
    cleaned_text = ''.join(cleaned_text)

    return cleaned_text

**Очистка:**

In [176]:
maxlen = 0
# Preprocessing training tweets
cleaned_train_texts = []
for tweet_text in train_tweet_texts:
    cleaned_text = clean_text(tweet_text).lower()
    split_cleaned_text = cleaned_text.split()
    # Estimating max length of all training tweets in tokens
    if len(split_cleaned_text) > maxlen:
        maxlen = len(split_cleaned_text)
    cleaned_train_texts.append(" ".join(split_cleaned_text))
    
# Preprocessing test tweets
cleaned_test_texts = []
for tweet_text in test_tweet_texts:
    cleaned_text = clean_text(tweet_text)
    cleaned_test_texts.append(" ".join(cleaned_text.split()))
    
# Preprocessing validation tweets
cleaned_dev_texts = []
for tweet_text in dev_tweet_texts:
    cleaned_text = clean_text(tweet_text)
    cleaned_dev_texts.append(" ".join(cleaned_text.split()))

**Лемматизация:**

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [178]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [179]:
def lemm(texts):
    texts = list(map(nltk.word_tokenize, texts))
    texts = list(map(lambda ww: [lemmatizer.lemmatize(w) for w in ww], texts))
    texts = list(map(lambda ww: [w for w in ww if w not in stop_words], texts))
    texts = [' '.join(ww) for ww in texts]
    return texts

In [ ]:
cleaned_train_texts

In [181]:
cleaned_train_texts = lemm(cleaned_train_texts)
cleaned_test_texts = lemm(cleaned_test_texts)
cleaned_dev_texts = lemm(cleaned_dev_texts)

# **Эмбеддинг:**

In [188]:
tokenizer = Tokenizer(lower=True, char_level=False)
tokenizer.fit_on_texts(cleaned_train_texts + cleaned_test_texts + cleaned_dev_texts)
# Converting texts to lists of ids
word_seq_train = tokenizer.texts_to_sequences(cleaned_train_texts)
word_seq_test = tokenizer.texts_to_sequences(cleaned_test_texts)
word_seq_dev = tokenizer.texts_to_sequences(cleaned_dev_texts)
word_index = tokenizer.word_index

# Padding too short tweet texts with '0's
word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=maxlen)
word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=maxlen)
word_seq_dev = sequence.pad_sequences(word_seq_dev, maxlen=maxlen)

dictionary_size = len(word_index.keys())
# 0-th token of embedding matrix is a padding token
embedding_matrix = np.zeros((dictionary_size + 1, EMBEDDINGS_DIM))\

vector_count = 0
for word, i in word_index.items():
    embedding_vector = fasttext_model.get_word_vector((word))
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        vector_count += 1
        embedding_matrix[i] = embedding_vector

In [196]:
print('dictionary coverage: ' + str(vector_count * 100 / len(word_index.items())) + '%') # not right, but 100% anyway

dictionary coverage: 100.0%


# **Компиляция модели:**

In [183]:
model = TextCNN(maxlen, dictionary_size + 1, EMBEDDINGS_DIM, embedding_weights=embedding_matrix)
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'], )
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3 , mode='max', restore_best_weights=True)

# **Обучение модели:**

In [184]:
model.fit(word_seq_train, train_labels,
              batch_size=BATCH_SIZE,
              epochs=CLASSIFIER_TRAIN_EPOCHS,
              callbacks=[early_stopping, ],
              validation_data=(word_seq_dev, dev_labels))

Epoch 1/10
67/67 [==============================] - 31s 459ms/step - loss: 0.2755 - accuracy: 0.9108 - val_loss: 0.2120 - val_accuracy: 0.9233
Epoch 2/10
67/67 [==============================] - 31s 458ms/step - loss: 0.1474 - accuracy: 0.9429 - val_loss: 0.2048 - val_accuracy: 0.9223
Epoch 3/10
67/67 [==============================] - 31s 458ms/step - loss: 0.0790 - accuracy: 0.9771 - val_loss: 0.2092 - val_accuracy: 0.9275
Epoch 4/10
67/67 [==============================] - 30s 454ms/step - loss: 0.0375 - accuracy: 0.9942 - val_loss: 0.2196 - val_accuracy: 0.9244
Epoch 5/10
67/67 [==============================] - 30s 450ms/step - loss: 0.0175 - accuracy: 0.9988 - val_loss: 0.2438 - val_accuracy: 0.9254
Epoch 6/10
67/67 [==============================] - 30s 450ms/step - loss: 0.0095 - accuracy: 0.9993 - val_loss: 0.2558 - val_accuracy: 0.9233


# **Вывод результатов:**

In [185]:
predicted_test_prob = model.predict(word_seq_test)
predicted_test_labels = []
predicted_dev_prob = model.predict(word_seq_dev)
predicted_dev_labels = []

for subarray in predicted_test_prob:
    label = 1 if subarray[0] >= CLASSIFICATION_THRESHOLD else 0
    predicted_test_labels.append(label)

for subarray in predicted_dev_prob:
    label = 1 if subarray[0] >= CLASSIFICATION_THRESHOLD else 0
    predicted_dev_labels.append(label)

In [186]:
dev_precision = precision_score(dev_labels, predicted_dev_labels, )
dev_recall = recall_score(dev_labels, predicted_dev_labels, )
dev_f_measure = f1_score(dev_labels, predicted_dev_labels, )
print(f"Dev:\nPrecision: {dev_precision}\n"
        f"Recall: {dev_recall}\nF-measure: {dev_f_measure}")

test_precision = precision_score(test_labels, predicted_test_labels, )
test_recall = recall_score(test_labels, predicted_test_labels, )
test_f_measure = f1_score(test_labels, predicted_test_labels, )
print(f"Test:\nPrecision: {test_precision}\n"
        f"Recall: {test_recall}\nF-measure: {test_f_measure}\n")


Dev:
Precision: 0.6666666666666666
Recall: 0.3170731707317073
F-measure: 0.4297520661157025
Test:
Precision: 0.723404255319149
Recall: 0.3434343434343434
F-measure: 0.4657534246575343



In [187]:
print(classification_report(test_labels, predicted_test_labels, digits=5))

              precision    recall  f1-score   support

           0    0.90780   0.98009   0.94256      1306
           1    0.72340   0.34343   0.46575       198

    accuracy                        0.89628      1504
   macro avg    0.81560   0.66176   0.70416      1504
weighted avg    0.88353   0.89628   0.87979      1504

